In [4]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [5]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [16,64]

In [6]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [7]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [8]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [9]:
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [10]:
df_ma.dropna(inplace=True)

In [11]:
df_ma.reset_index(drop=True, inplace=True)

In [12]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2023-06-15T20:00:00.000000000Z,1.09476,1.09492,1.09452,1.09455,1.088339,1.082778
1,2023-06-15T21:00:00.000000000Z,1.09444,1.09456,1.09422,1.09430,1.089117,1.083028
2,2023-06-15T22:00:00.000000000Z,1.09439,1.09496,1.09439,1.09491,1.089983,1.083272
3,2023-06-15T23:00:00.000000000Z,1.09494,1.09514,1.09452,1.09455,1.090723,1.083500
4,2023-06-16T00:00:00.000000000Z,1.09456,1.09508,1.09400,1.09420,1.091405,1.083710


In [13]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64

In [14]:
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)

In [15]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return True
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return True
    return False

In [16]:
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)

In [17]:
df_trades = df_ma[df_ma.IS_TRADE==True].copy()

In [18]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
49,2023-06-19T21:00:00.000000000Z,1.09226,1.09248,1.09206,1.09229,1.092157,1.092384,-0.000227,0.000044,True
92,2023-06-21T16:00:00.000000000Z,1.09506,1.09738,1.09497,1.09706,1.092537,1.092304,0.000234,-0.000055,True
129,2023-06-23T05:00:00.000000000Z,1.09331,1.09341,1.09273,1.09326,1.095028,1.095062,-0.000035,0.000294,True
178,2023-06-27T06:00:00.000000000Z,1.09176,1.09336,1.09162,1.09311,1.091456,1.091286,0.000170,-0.000018,True
216,2023-06-28T20:00:00.000000000Z,1.09161,1.09170,1.09113,1.09128,1.093011,1.093067,-0.000056,0.000184,True


In [22]:
df_plot = df_ma.iloc[25:75].copy()

In [23]:
df_plot.shape

(50, 10)

In [24]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()